In [ ]:
#!/usr/bin/env python
# coding: utf-8
get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 

import gc
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
print(os.getcwd())

base_path = '.'

df_train = pd.read_csv(os.path.join(base_path , 'input/train.csv'), index_col=0)
df_test = pd.read_csv(os.path.join(base_path , 'input/public_test.csv'), index_col=0)
df_test['smishing'] = -1

df_fea = pd.concat([df_train, df_test])
df_fea.shape

### Mecab

In [ ]:
# mecab = Mecab()
# df_fea['morphs'] = df_fea['text'].apply(lambda x: mecab.morphs(x))
# df_fea['morphs_str'] = df_fea['morphs'].apply(lambda x: ' '.join(x))
# df_fea['nouns'] = df_fea['text'].apply(lambda x: mecab.nouns(x))
# df_fea['nouns_str'] = df_fea['nouns'].apply(lambda x: ' '.join(x))

# def pos(row):
#     x = row['text']
#     pos_dict = {c:0 for c in pos_cols}
    
#     for _, p in mecab.pos(x):
#         for v in p.split('+'):
#             pos_dict[v] += 1
        
#     return [pos_dict[k] for k in sorted(pos_dict.keys())]

# pos_cols = [
#     'EC',
#     'EF',
#     'EP',
#     'ETM',
#     'ETN',
#     'IC',
#     'JC',
#     'JKB',
#     'JKC',
#     'JKG',
#     'JKO',
#     'JKQ',
#     'JKS',
#     'JKV',
#     'JX',
#     'MAG',
#     'MAJ',
#     'MM',
#     'NA',
#     'NNB',
#     'NNBC',
#     'NNG',
#     'NNP',
#     'NP',
#     'NR',
#     'SC',
#     'SF',
#     'SL',
#     'SN',
#     'SSC',
#     'SSO',
#     'SY',
#     'UNA',
#     'UNKNOWN',
#     'VA',
#     'VCN',
#     'VCP',
#     'VV',
#     'VX',
#     'XPN',
#     'XR',
#     'XSA',
#     'XSN',
#     'XSV',
# ]
# df_fea[pos_cols] = df_fea.apply(pos, axis=1, result_type='expand')
# df_fea[pos_cols] = df_fea[pos_cols].astype(np.int16)
# df_fea.to_pickle('data/df_fea_1.pkl')

In [ ]:
df_fea = pd.read_pickle('data/df_fea_1.pkl')
df_fea.head(10)

In [ ]:
df_fea.columns

In [ ]:
dfs = []
fea_dict = {}

# vocab = None
# stop_words = joblib.load('model/vocab_20191231T111937.pkl')

vocab = joblib.load('model/vocab_20191231T111937.pkl')
stop_words = []

### Seq Fea

In [ ]:
def _toidx(x):
    return [int(vocab_dict[x[i]]) if i < len(x) else 0 for i in range(max_len)]

def toidx(src_col, max_len):
    idx_col_nm = '{}_{}_idx'.format(src_col, max_len)

    vocab_set = set()
    _ = df_fea[src_col].apply(lambda x: [vocab_set.add(c) for c in x])

    vocab_dict = {v: i+1 for i, v in enumerate(vocab_set)}
    vocab_dim = len(vocab_dict.keys()) + 1

    def toidx(x):
        return [int(vocab_dict[x[i]]) if i < len(x) else 0 for i in range(max_len)]

#     print(vocab_dim, max_len)
    
    return idx_col_nm, df_fea[src_col].apply(toidx), vocab_dim, max_len

In [ ]:
src_col = 'morphs'
max_len = 128

c, d, vocab_dim, max_len = toidx(src_col, max_len)
df_fea[c] = d
c, vocab_dim, max_len

In [ ]:
# src_col = 'nouns'
# max_len = 100

# c, d, vocab_dim, max_len = toidx(src_col, max_len)
# df_fea[c] = d
# c

### Basic Fea

In [ ]:
df_fea['fea__text_len'] = df_fea['text'].str.len().fillna(0).astype(np.float16)
df_fea['fea__morphs_cnt'] = df_fea['morphs'].apply(lambda x: len(x)).fillna(0).astype(np.float16)
df_fea['fea__noun_cnt'] = df_fea['nouns'].apply(lambda x: len(x)).fillna(0).astype(np.float16)

### Count

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='char',
#                              vocabulary=vocab,
                             stop_words=stop_words, 
                             max_df=1.0,
                             min_df=100)

# vectorizer = vectorizer.fit(df_fea[df_fea['smishing']==1]['nouns_str'].values)
vectorizer = vectorizer.fit(df_fea['text'].values)
cnt_vec = vectorizer.transform(df_fea['text'].values).toarray()

cnt_dict = {'cnt_{0:04d}'.format(i):'cnt_{0:04}_{1}'.format(i, c) for i, c in enumerate(vectorizer.get_feature_names())}
fea_dict.update(cnt_dict)
cnt_cols = sorted(cnt_dict.keys())

df_cnt_vec = pd.DataFrame(data=cnt_vec, index=df_fea.index, columns=cnt_cols, dtype=np.float16)
df_cnt_vec = df_cnt_vec.loc[:, (df_cnt_vec != 0).any(axis=0)]
dfs.append(df_cnt_vec)
df_cnt_vec.shape

### tfidf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf(data, params, tfidf_tag):
    vectorizer = TfidfVectorizer(**params)
    vectorizer = vectorizer.fit(data)

    d = {'{0}_{1:04d}'.format(tfidf_tag, v):'{0}_{1:04d}_{2}'.format(tfidf_tag, v, k) for k, v in sorted(vectorizer.vocabulary_.items(), key=lambda item: item[1])}
    c = sorted(d.keys())

    df = pd.DataFrame(
#         data=vectorizer.transform(df_fea['nouns_str'].values).toarray(),
        data=vectorizer.transform(df_fea['morphs_str'].values).toarray(),
        columns=c, 
        index=df_fea.index,
        dtype=np.float16)
    
    # Remove all zeros column
    df = df.loc[:, (df != 0).any(axis=0)]
    d = {k:v for k, v in d.items() if k in df.columns}
    
    print(tfidf_tag, df.shape)
    
    return df, d, vectorizer

In [ ]:
tfidf_src_col = 'morphs_str'

#### 1, 1

##### char

In [ ]:
params = {
    'analyzer' : 'char',
    'max_features':None, 
#     'vocabulary': vocab,
    'stop_words':stop_words, 
#     'max_df':0.1, 
    'min_df':100, 
    'ngram_range':(1, 1), 
}
df, d, v = tfidf(df_fea[tfidf_src_col].values, params, 'tfidf_char_11')
dfs.append(df)
fea_dict.update(d)

##### word

In [ ]:
params = {
    'max_features':2000, 
#     'vocabulary': vocab,
    'stop_words':stop_words, 
    'max_df':1.0, 
    'min_df':200, 
    'ngram_range':(1, 1), 
}
df, d, v = tfidf(df_fea[tfidf_src_col].values, params, 'tfidf_word_11')
dfs.append(df)
fea_dict.update(d)

#### 2, 2

In [ ]:
params = {
    'max_features':2000, 
#     'vocabulary': vocab,
    'stop_words':stop_words, 
    'max_df':1.0, 
    'min_df':500, 
    'ngram_range':(2, 2), 
}
df, d, v = tfidf(df_fea[tfidf_src_col].values, params, 'tfidf_word_22')
dfs.append(df)
fea_dict.update(d)

#### 3, 3

In [ ]:
params = {
    'max_features':500, 
#     'vocabulary': vocab,
    'stop_words':stop_words, 
    'max_df':1.0, 
    'min_df':500, 
    'ngram_range':(3, 3), 
}
df, d, v = tfidf(df_fea[tfidf_src_col].values, params, 'tfidf_word_33')
dfs.append(df)
fea_dict.update(d)

#### smishing 1, 1

##### char

In [ ]:
params = {
    'analyzer' : 'char',
#     'max_features':500, 
#     'vocabulary': vocab,
    'stop_words':stop_words, 
#     'max_df':1.0, 
#     'min_df':100, 
    'ngram_range':(1, 1), 
}
df, d, v = tfidf(df_fea[df_fea['smishing']==1][tfidf_src_col].values, params, 'tfidf_pos_char_11')
dfs.append(df)
fea_dict.update(d)


##### word

In [ ]:
params = {
    'max_features':500, 
#     'vocabulary': vocab,
    'stop_words':stop_words, 
    'max_df':1.0, 
    'min_df':100, 
    'ngram_range':(1, 1), 
}
df, d, v = tfidf(df_fea[df_fea['smishing']==1][tfidf_src_col].values, params, 'tfidf_pos_word_11')
dfs.append(df)
fea_dict.update(d)


#### smishing 2, 2

In [ ]:
params = {
    'max_features':500, 
#     'vocabulary': vocab,
    'stop_words':stop_words, 
    'max_df':1.0, 
    'min_df':100, 
    'ngram_range':(2, 2), 
}
df, d, v = tfidf(df_fea[df_fea['smishing']==1][tfidf_src_col].values, params, 'tfidf_pos_word_22')
dfs.append(df)
fea_dict.update(d)


### W2V

In [ ]:
# ts = '20191214T055747'
# file_name = 'model/gensim_{}'.format(ts)

# w2v_model = joblib.load(os.path.join(base_path, '{}.pkl'.format(file_name)))
# w2v_size = w2v_model.wv.vectors.shape[1]

# def mean_w2v(row):
#     nouns = row['nouns']
#     w2v = np.zeros(w2v_size)

    
#     for n in nouns:
#         if n in w2v_model.wv.vocab.keys():
#             w2v = np.add(w2v, w2v_model.wv[n])
            
#     return w2v if len(nouns) == 0 else np.true_divide(w2v, len(nouns))

# w2v_cols = ['w2v_{}'.format(i) for i in range(w2v_size)]

# df_fea[w2v_cols] = df_fea.apply(mean_w2v, axis=1, result_type='expand')

# for c in w2v_cols:
#     df_fea[c] = df_fea[c].astype(np.float16)

### Merged

In [ ]:
print(df_fea.shape)
for df in dfs:
    print(df.shape)

df_merged = pd.concat([df_fea] + dfs, axis=1)
print('df_merged', df_merged.shape)
print(df_merged.info())
# df_merged.head(10)


In [ ]:
idx_cols = ['id', 'year_month', 'text', 'smishing', 'nouns', 'nouns_str', 'morphs', 'morphs_str']

cat_cols = []
fea_cols = [c for c in df_merged.columns if c not in idx_cols]

for c in fea_cols:
    if c not in fea_dict.keys():
        fea_dict[c] = c

len(fea_cols), len(fea_dict.keys()), vocab_dim, max_len

#### MinMaxScaler

In [ ]:
del df_fea, df_train, df_test
for df in dfs:
    del df
gc.collect()

In [ ]:
# base_dict = (df_merged[fea_cols].max() - df_merged[fea_cols].min()).to_dict()

# for c in fea_cols:
#     df_merged[c] = df_merged[c] / base_dict[c]

# print(df_merged[fea_cols].info())

In [ ]:
# merged_ts = datetime.now().strftime('%Y%m%dT%H%M%S') + '_' + str(len(fea_cols))
merged_ts = '{}_{}_{}_{}'.format(datetime.now().strftime('%Y%m%dT%H%M%S'), 
                                 str(len(fea_cols)), 
                                 str(max_len), 
                                 str(vocab_dim))
print(merged_ts)
for c in df_merged:
    if c not in fea_cols + ['smishing']:
        df_merged.drop(c, axis=1, inplace=True)
        

In [ ]:
df_merged.info()

### Dump

In [ ]:
joblib.dump(fea_dict, 'data/df_merged_{}_fea_dict.pkl'.format(merged_ts))

In [ ]:
joblib.dump(df_merged.loc[df_merged['smishing'] != -1,:], 'data/df_merged_{}_train.pkl'.format(merged_ts))

In [ ]:
joblib.dump(df_merged.loc[df_merged['smishing'] == 1,:], 'data/df_merged_{}_train_pos.pkl'.format(merged_ts))

In [ ]:
joblib.dump(df_merged.loc[df_merged['smishing'] == -1,:], 'data/df_merged_{}_test.pkl'.format(merged_ts))

In [ ]:
# del df_merged, df_fea, df_train, df_test
# for df in dfs:
#     del df

In [ ]:
gc.collect()

In [ ]:
exit(0)